In [32]:
# 1. Create a dataset
#   1.1 Cosmus dataset
#   1.2 Get only Ukr comments
#   1.3 Find out dataset quality
# 2. Show data contribution
# 3. Get model with pretrained values
# 4. Convert dataset to way it suppose to be eaten by model
# 5. model training with training data part
# 6. Learning graphic
# 7. Check results with validation data part

In [33]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import pandas as pd
from matplotlib import pyplot as plt

import numpy as np
import evaluate

In [2]:
# 1. Завантаження датасету
ds = load_dataset("YShynkarov/COSMUS")

In [3]:
# Якщо лише спліт “train”, можеш розділити:
if "validation" not in ds:
    ds = ds["train"].train_test_split(test_size=0.2)
    ds = DatasetDict({"train": ds["train"], "validation": ds["test"]})
else:
    # або якщо вже є validation
    pass

In [58]:
df = ds["train"].to_pandas()
#print(df.head())
print(df["gpt_labels_v1"].unique())

['negative' 'positive' 'neutral' "Output: 'negative'" "'negative'" 'mixed'
 "Output: 'positive'" "Output: 'mixed'" "'positive'" 'negative.'
 'Output: negative' 'Output: positive' "Output: 'neutral'" "'neutral'"
 'Звісно, напишіть текст, який ви бажаєте проаналізувати на настрій!'
 'neutral  ' 'Output: mixed' "1. 'negative'\n2. 'negative'\n3. 'neutral'"
 'negative '
 'Моя мова зараз українська. Я завжди готова допомогти! Як я можу вам сьогодні допомогти?'
 'mixed '
 'Input: "Києве продовжуються вибухи та відключення світла в різних районах міста. Жителі скаржаться на невпевненість у майбутньому."\nOutput: \'negative\''
 'neutral   ']


In [11]:
# 2. Визначення міток
labels = ["positive", "negative", "neutral", "mixed"]
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for i, l in enumerate(labels)}


{'positive': 0, 'negative': 1, 'neutral': 2, 'mixed': 3}



In [20]:

# 3. Токенайзер + препроцесінг
model_name = "xlm-roberta-base"  # або інша модель, яку будеш використовувати
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):

    label_str = example["gpt_labels_v1"].strip()       # прибрати пробіли
    label_str = label_str.replace("'", "").replace('"', "")  # прибрати лапки
    label_str = label_str.lower()                      # привести до нижнього регістру

    if label_str not in label2id:
    # якщо є неочікувані значення — можна пропустити або присвоїти 'neutral'
        label_str = "neutral"

    example_enc = tokenizer(
        example["document_content"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    example_enc["labels"] = label2id[label_str]

    return example_enc

In [22]:
from datasets import Value

encoded = ds.map(preprocess, batched=False)
encoded = encoded.remove_columns([c for c in ds["train"].column_names if c not in ["input_ids", "attention_mask", "labels"]])

#encoded = encoded.cast_column("labels", encoded["train"].features["labels"].dtype)
encoded = encoded.cast_column("labels", Value("int64"))

encoded.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Casting the dataset: 100%|██████████| 2445/2445 [00:00<00:00, 87671.93 examples/s]


In [23]:

# 4. Створення моделі
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    problem_type="single_label_classification",
    id2label=id2label,
    label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:

# 5. Метрика
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

In [29]:

# 6. Аргументи тренування
training_args = TrainingArguments(
    output_dir="cosmus_sentiment_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [30]:

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded["train"],
    eval_dataset=encoded["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\Denys\AppData\Local\Temp\ipykernel_10920\4015940683.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:

# 8. Тренування
trainer.train()
trainer.save_model("cosmus_sentiment_model")
tokenizer.save_pretrained("cosmus_sentiment_model")

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.859900,0.700872,0.764417,0.743043


('cosmus_sentiment_model\\tokenizer_config.json',
 'cosmus_sentiment_model\\special_tokens_map.json',
 'cosmus_sentiment_model\\tokenizer.json')

In [67]:
logs = pd.DataFrame(trainer.state.log_history)
logs.head()

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_accuracy,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,0.8599,33.980572,0.000004,0.816993,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.000000,612,0.700872,0.764417,0.743043,280.8661,8.705,0.545,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.000000,612,NaN,NaN,NaN,NaN,NaN,NaN,5759.3428,1.698,0.106,6.432523e+14,0.826824


In [68]:
from datasets import Value


encoded_test = ds.map(preprocess, batched=False)
encoded_test = encoded_test.remove_columns([c for c in ds["validation"].column_names if c not in ["input_ids", "attention_mask", "labels"]])

#encoded = encoded.cast_column("labels", encoded["train"].features["labels"].dtype)
encoded_test = encoded_test.cast_column("labels", Value("int64"))

encoded_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

results = trainer.evaluate(encoded_test)
print(results)

C:\Users\Denys\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Denys\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_train_loss': 0.5770187973976135, 'eval_train_accuracy': 0.8124552612741589, 'eval_train_f1': 0.7950776984683634, 'eval_train_runtime': 1247.4061, 'eval_train_samples_per_second': 7.839, 'eval_train_steps_per_second': 0.491, 'epoch': 1.0, 'eval_validation_loss': 0.7008722424507141, 'eval_validation_accuracy': 0.7644171779141105, 'eval_validation_f1': 0.743042927650663, 'eval_validation_runtime': 328.4605, 'eval_validation_samples_per_second': 7.444, 'eval_validation_steps_per_second': 0.466}


In [66]:

# 9. Тест через pipeline
from transformers import pipeline
clf = pipeline("text-classification", model="cosmus_sentiment_model", tokenizer="cosmus_sentiment_model")

texts = [
    "Я дуже радий цьому!",
    "Мені страшно за майбутнє.",
    "Це найгірший день у моєму житті.",
    "не ну такого я ще не бачив",
    "Отримав провід — працює, але горить червоним індикатором, не зрозуміло чому. Думав ще один замовити, але вже не впевнений",
    "цей кабель не працює з моїм павербанком ксяеми на 5000 ма, Ну якось так"
]

results = clf(texts)
for t, r in zip(texts, results):
    print(f"{t} → {r['label']} ({r['score']:.2f})")


Device set to use cpu


Я дуже радий цьому! → positive (0.91)
Мені страшно за майбутнє. → negative (0.87)
Це найгірший день у моєму житті. → negative (0.86)
не ну такого я ще не бачив → negative (0.82)
Отримав провід — працює, але горить червоним індикатором, не зрозуміло чому. Думав ще один замовити, але вже не впевнений → negative (0.91)
цей кабель не працює з моїм павербанком ксяеми на 5000 ма, Ну якось так → negative (0.91)
